# Trading Bot

In [11]:
from datetime import datetime
import os
import time

from apscheduler.schedulers.background import BackgroundScheduler
import pandas as pd
import pandas_ta as ta

from utilities.charting import plot_candlesticks_RSI_chart
from utilities.getData_YahooFinanace import update_daily_data, update_60m_data, update_5m_data
from utilities.telegram_alerts import send_message_to_telegram_group

from strategies.signal_generators import generate_rsi_ema_long_short_signals, generate_exit_signals


In [12]:
# Set Global Variables
tickers = ["RELIANCE.NS","AAPL","ADANIPORTS.NS","^NSEI"]

### ⚙️ Data Update + Indicator Calculation

This function:
- Updates raw daily and intraday data
- Computes RSI and EMA indicators using `pandas_ta`
- Saves the output to `data/processed/`

In [13]:
def update_data_and_calculate_indicators(tickers):
    # Update data
    update_daily_data(tickers)
    update_60m_data(tickers)

    for ticker in tickers:
        try:
            # === 60m Data Processing ===
            df_60m = pd.read_csv(f"data/raw/{ticker}_60m.csv", parse_dates=['Datetime'], index_col='Datetime')
            df_60m.ta.ema(length=20, append=True)
            df_60m.ta.ema(length=200, append=True)
            df_60m.ta.rsi(length=14, append=True)
            df_60m.ta.rsi(length=8, append=True)
            df_60m.to_csv(f"data/indicators/{ticker}_60m.csv")
            print(f"✅ 60m indicators added for {ticker}")

            # === Daily Data Processing ===
            df_daily = pd.read_csv(f"data/raw/{ticker}_D.csv", parse_dates=['Date'], index_col='Date')
            df_daily.ta.ema(length=20, append=True)
            df_daily.ta.ema(length=200, append=True)
            df_daily.ta.rsi(length=14, append=True)
            df_daily.ta.rsi(length=8, append=True)
            df_daily.to_csv(f"data/indicators/{ticker}_D.csv")
            print(f"✅ Daily indicators added for {ticker}")

        except Exception as e:
            print(f"❌ Error processing {ticker}: {e}")

### Generate Signals

In [14]:
def run_signals_generator(tickers):
    """
    Processes a list of tickers to generate trading signals based on RSI and EMA crossover strategy.

    For each ticker:
    - Loads the 60-minute indicators data from 'data/indicators/{ticker}_60m.csv'.
    - Applies the RSI-EMA signal generator.
    - Saves the resulting DataFrame with signals to 'data/signals/{ticker}_60m.csv'.

    Parameters:
        ticker_list (list of str): List of ticker symbols to process.

    Returns:
        None
    """
    os.makedirs("data/signals", exist_ok=True)  # Ensure output directory exists

    for ticker in tickers:
        try:
            file_path = f"data/indicators/{ticker}_60m.csv"
            df = pd.read_csv(file_path, parse_dates=['Datetime'], index_col='Datetime')

            # Step 1: Generate Entry Signals
            df = generate_rsi_ema_long_short_signals(df, rsi_column='RSI_8', ema_column='EMA_20')

            # Step 2: Generate Exit Signals
            df = generate_exit_signals(df, rsi_column='RSI_8', entry_signal_column='Entry Signal')

            # Step 3: Save the DataFrame with both entry & exit signals
            output_path = f"data/signals/{ticker}_60m.csv"
            df.to_csv(output_path)
            print(f"✅ 60m Signals (entry + exit) generated and saved for {ticker}")

        except Exception as e:
            print(f"❌ Failed for {ticker}: {e}")

### Send Alerts to Telegram

In [15]:
def send_alerts_to_telegram(tickers):
    """
    Sends Telegram alerts for:
    - New trade entries
    - Live MTM updates (every hour)
    - Trade exits (profit booked or stop-loss hit)

    Parameters:
        tickers (list): List of ticker symbols.
    """
    for ticker in tickers:
        try:
            file_path = f"data/signals/{ticker}_60m.csv"
            df = pd.read_csv(file_path, parse_dates=['Datetime'], index_col='Datetime')
            latest_row = df.iloc[-1]

            signal = latest_row.get("Entry Signal")
            trade_status = latest_row.get("Trade_Status")
            exit_reason = latest_row.get("Exit_Reason")
            alert_sent = latest_row.get("Alert_Sent")
            last_close = latest_row.get("Close")

            message = None

            # === Entry Alert ===
            if pd.notnull(signal) and pd.isnull(alert_sent):
                message = (
                    f"*Entry Alert*\n"
                    f"Stock: {ticker}\n"
                    f"Signal: {signal} 📊\n"
                    f"Strategy: RSI-EMA Confirmation\n"
                    f"Go {'📈 Long' if 'Long' in signal else '📉 Short'} at {last_close:.2f}"
                )

            # === Exit Alert ===
            elif trade_status == "Exited" and pd.isnull(alert_sent):
                message = (
                    f"*Exit Alert*\n"
                    f"Stock: {ticker}\n"
                    f"Status: {exit_reason} ✅\n"
                    f"Exit Price: {last_close:.2f}"
                )

            # === Live Trade Status Update ===
            elif isinstance(trade_status, str) and ("Long:" in trade_status or "Short:" in trade_status) and pd.isnull(alert_sent):
                message = (
                    f"⏱ *Live MTM Update*\n"
                    f"Stock: {ticker}\n"
                    f"Status: {trade_status.split(':')[0]}\n"
                    f"MTM: {trade_status.split(':')[1]}\n"
                    f"Last Price: {last_close:.2f}"
                )

            # Send and update if a message was created
            if message:
                send_message_to_telegram_group(message)
                df.at[df.index[-1], "Alert_Sent"] = datetime.now().strftime("%Y-%m-%d %H:%M")
                df.to_csv(file_path)
                print(f"✅ Alert sent for {ticker}")

        except Exception as e:
            print(f"❌ Failed to process {ticker}: {e}")

### Trading Bot Core Function

In [16]:
from IPython.display import clear_output

def run_trading_bot(tickers: list[str]) -> None:
    """
    Runs the full trading bot workflow:
    1. Updates market data and computes indicators.
    2. Generates trading signals.
    3. Sends alerts to Telegram for valid signals.

    Parameters:
        tickers (list[str]): List of stock tickers to process.
    """
    clear_output(wait=True)  # Clears the notebook cell output
    
    print(f"\n🕒 Trading Bot started at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

    try:
        # Step 1: Update data and calculate indicators
        update_data_and_calculate_indicators(tickers)
    except Exception as e:
        print(f"❌ Failed during data update: {e}")

    try:
        # Step 2: Generate signals
        run_signals_generator(tickers)
    except Exception as e:
        print(f"❌ Failed during signal generation: {e}")

    try:
        # Step 3: Send alerts to Telegram
        send_alerts_to_telegram(tickers)
    except Exception as e:
        print(f"❌ Failed during sending alerts: {e}")

### Scheduled Runs

In [17]:
def start_scheduler():
    """
    Initializes and starts the APScheduler to run the trading bot at the xxth minute of every hour.
    Keeps the script running indefinitely until interrupted.
    """
    scheduler = BackgroundScheduler()
    scheduler.add_job(run_trading_bot, 'cron', args=[tickers], minute=16, id='trading_bot_job', name='Hourly Trading Bot Job')
    scheduler.start()
    print("✅ APScheduler started. Trading bot will run at the 16th minute of every hour.")

    try:
        # Keep the script running
        while True:
            time.sleep(45)
    except (KeyboardInterrupt, SystemExit):
        print("⛔ Scheduler stopped.")
        scheduler.shutdown()

In [18]:
start_scheduler()


🕒 Trading Bot started at 2025-05-07 12:16:00
⬇️ Downloading RELIANCE.NS Daily data from 2025-05-06 to 2025-05-06...
YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


✅ RELIANCE.NS Daily data is already up to date. No new rows to add.
⬇️ Downloading AAPL Daily data from 2025-05-06 to 2025-05-06...


[*********************100%***********************]  1 of 1 completed


✅ AAPL Daily data is already up to date. No new rows to add.
⬇️ Downloading ADANIPORTS.NS Daily data from 2025-05-06 to 2025-05-06...


...

✅ AAPL 60m data is already up to date. No new rows added.
⬇️ Downloading ADANIPORTS.NS 60m data from 2025-05-07 10:15:00 to 2025-05-07 12:15:00...


[*********************100%***********************]  1 of 1 completed


✅ ADANIPORTS.NS 60m data updated with 1 new rows at 12:16:02.
⬇️ Downloading ^NSEI 60m data from 2025-05-07 10:15:00 to 2025-05-07 12:15:00...


[*********************100%***********************]  1 of 1 completed


✅ ^NSEI 60m data updated with 1 new rows at 12:16:02.
✅ 60m indicators added for RELIANCE.NS
✅ Daily indicators added for RELIANCE.NS
✅ 60m indicators added for AAPL
✅ Daily indicators added for AAPL
✅ 60m indicators added for ADANIPORTS.NS
✅ Daily indicators added for ADANIPORTS.NS
✅ 60m indicators added for ^NSEI
✅ Daily indicators added for ^NSEI
✅ 60m Signals (entry + exit) generated and saved for RELIANCE.NS
✅ 60m Signals (entry + exit) generated and saved for AAPL
✅ 60m Signals (entry + exit) generated and saved for ADANIPORTS.NS
✅ 60m Signals (entry + exit) generated and saved for ^NSEI
✅ Alert sent for RELIANCE.NS
✅ Alert sent for ADANIPORTS.NS
✅ Alert sent for ^NSEI
⛔ Scheduler stopped.


### Manual Runs

In [ ]:
# Run the complete Trading Bot Workflow
run_trading_bot(tickers)

In [ ]:
# Update the data and calculate indicators
update_data_and_calculate_indicators(tickers)

In [ ]:
# Update the Daily data
update_daily_data(tickers)

In [ ]:
# Update the 60m data
update_60m_data(tickers)

### Charts

In [ ]:
df = pd.read_csv("data/processed/RELIANCE.NS_60m.csv", parse_dates=['Datetime'], index_col='Datetime').tail(200)

In [ ]:
plot_candlesticks_RSI_chart(df)

In [ ]:
plot_candlesticks_RSI_chart(df, 8)